<a href="https://colab.research.google.com/github/sanjay-ms8/hello-world/blob/main/U_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

def down_block(x, filters, kernel_size=(3, 3), padding="same", activation="relu"):
    x = layers.Conv2D(filters, kernel_size, padding=padding, activation=activation)(x)
    x = layers.Conv2D(filters, kernel_size, padding=padding, activation=activation)(x)
    x_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(x)
    return x_pool, x

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", activation="relu"):
    x = layers.Conv2DTranspose(filters, (2, 2), strides=(2, 2), padding="same")(x)
    x = layers.concatenate([x, skip])
    x = layers.Conv2D(filters, kernel_size, padding=padding, activation=activation)(x)
    x = layers.Conv2D(filters, kernel_size, padding=padding, activation=activation)(x)
    return x

def build_unet(input_shape, num_filters=16, activation="relu"):
    inputs = layers.Input(input_shape)

    # Downsampling through the network
    x_pool, x = down_block(inputs, num_filters)
    x_pool, x = down_block(x_pool, num_filters * 2)
    x_pool, x = down_block(x_pool, num_filters * 4)

    # Bottleneck layer
    bottleneck = layers.Conv2D(num_filters * 8, (3, 3), padding="same", activation=activation)(x_pool)

    # Upsampling and reconstruction
    x = up_block(bottleneck, x, num_filters * 4)
    x = up_block(x, x, num_filters * 2)
    x = up_block(x, x, num_filters)

    # Output layer
    outputs = layers.Conv2D(1, (1, 1), activation="sigmoid")(x)

    # Model definition
    model = tf.keras.Model(inputs, outputs)
    return model

input_shape = (256, 256, 1)
model = build_unet(input_shape)
model.compile(optimizer="adam", loss="binary_crossentropy")


ValueError: ignored